In [1]:
# Install the package
!pip install easy_es

In [2]:
import pandas as pd

from easy_es import EventStudy, plot_mean_car, calculate_car_stats, plot_joint_mean_car

### Run single event study

In [3]:
# Load test events
events_df = pd.read_csv('https://raw.githubusercontent.com/Darenar/easy-event-study/main/tests/test_one/input_events.csv')

In [4]:
# Initialize EventStudy class object
event_study = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5',
    n_cores=1 # Change for multiprocessing
)

# Run the study
results_df = event_study.run_study(events_df)

100%|██████████| 33/33 [00:17<00:00,  1.85it/s]


In [5]:
# Calculate average results in a dataframe format
calculate_car_stats(results_df, critical_value=0.95)

,mean,median,sd,t_stat,p_value,upper_ci,lower_ci
Offset,,,,,,,
-10,-0.005119,-0.003075,0.002696,-1.898441,0.058,0.000166,-0.010403
-9,-0.006913,-0.005001,0.003902,-1.771815,0.076,0.000734,-0.014561
-8,-0.008918,-0.003972,0.006029,-1.479152,0.139,0.002899,-0.020736
-7,-0.009972,-0.004459,0.006938,-1.437230,0.151,0.003627,-0.023571
-6,-0.014504,-0.002461,0.007390,-1.962646,0.050,-0.000020,-0.028989
-5,-0.008355,0.002768,0.008282,-1.008806,0.313,0.007877,-0.024587
-4,-0.008015,0.002879,0.009823,-0.815918,0.415,0.011238,-0.027267
-3,-0.008249,0.003756,0.010280,-0.802498,0.422,0.011898,-0.028397
-2,-0.005908,-0.001566,0.010336,-0.571537,0.568,0.014351,-0.026167


In [6]:
# Plot graph with the results
plot_mean_car(results_df, critical_value=0.9)

### Pre-loading returns data

In [7]:
### One can explicitely load the returns data and to dump into a CSV file 
event_study.add_returns(
    list_of_tickers=events_df['ticker'].unique(),
    min_date=events_df['event_date'].min(),
    max_date=events_df['event_date'].max()
)
event_study.returns_df.to_csv('returns_test_dump.csv', index=False)

100%|██████████| 33/33 [00:18<00:00,  1.79it/s]


In [8]:
# Then to reload it and initialize the event-study with pre-loaded returns
event_study = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5'
)
event_study.add_returns(ret_df=pd.read_csv('returns_test_dump.csv'))

# Run the study
results_df = event_study.run_study(events_df)

### Plotting two event study results together

In [9]:
# Run first event study
events_df_one = pd.read_csv('https://raw.githubusercontent.com/Darenar/easy-event-study/main/tests/test_one/input_events.csv')
event_study_one = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5'
)
result_one = event_study_one.run_study(events_df_one)

100%|██████████| 33/33 [00:20<00:00,  1.62it/s]


In [10]:
# Run second event study
events_df_two = pd.read_csv('https://raw.githubusercontent.com/Darenar/easy-event-study/main/tests/test_two/input_events.csv')
event_study_two = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5'
    
)
result_two = event_study_two.run_study(events_df_two)

100%|██████████| 41/41 [00:25<00:00,  1.62it/s]


In [11]:
plot_joint_mean_car(
    result_one, 
    result_two, 
    name_one='Increase', 
    name_two='Decrease')

### Bootstrap test

The confidence intervals around mean CAR estimates are calculated using the parametric method.

One also could calculate them using non-parametric approach - by bootstrap. Strategy:
1. Given a list of ticker and events, randomly permute the events N times.
2. Calculate mean CAR estimate for each of these N-runs.
3. Calculate confidence by using quantiles (e.g. 0.05 and 0.95) on those N runs.
4. Plot together with the original mean CAR to see the significance.


Note: highly depends on the number of bootstrap runs. If possible, increase **n_cores** parameter to the greater number of cpu in use

In [12]:
# For bootstrap - if there are several CPU cores available - it is better to use them 
import psutil
CPU_CORES = psutil.cpu_count()
print(f"{CPU_CORES} cores available")

10 cores available


In [13]:
# Initialize EventStudy class object
event_study = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5',
    n_cores=CPU_CORES
)

# Run the study
results_df = event_study.run_study(events_df)

  0%|          | 0/33 [00:00<?, ?it/s]

100%|██████████| 33/33 [00:18<00:00,  1.77it/s]


In [14]:
bootstrap_results_df = event_study.run_bootstrap_study(events_df, n_bootstrap=100)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 2679.47it/s]


In [15]:
# Each column represents mean CAR value obtained in each bootstrap run
bootstrap_results_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
Offset,,,,,,,,,,,,,,,,,,,,,
-10,0.002857,-0.000930,-0.000569,-0.003973,-0.003516,-0.000342,-0.001367,-0.003918,-0.000704,0.006883,...,0.001598,-0.004992,0.003323,-0.001014,-0.002297,0.001953,0.001289,-0.002962,0.004557,0.000752
-9,0.001295,0.002066,0.001748,-0.007525,-0.001740,0.005547,-0.006788,0.006664,0.000416,0.012453,...,0.002173,-0.002083,0.009599,0.001136,0.005194,-0.000528,-0.001880,0.006051,0.008829,0.015905
-8,-0.003024,0.000249,0.001258,-0.008464,-0.002183,0.009989,-0.008308,0.005507,-0.001744,0.017194,...,0.001197,-0.001283,0.004516,0.001970,0.003832,-0.003268,-0.004967,0.006418,0.007339,0.010300
-7,0.000187,-0.003541,0.008272,-0.008078,0.003852,0.008433,-0.007332,0.008599,0.002666,0.016530,...,0.006014,-0.001305,-0.000128,0.007666,-0.002024,-0.005005,-0.005124,0.007321,0.003408,0.015932
-6,0.002124,-0.004338,0.014155,-0.007725,0.008104,0.008553,-0.001250,0.010539,0.004928,0.022408,...,0.000351,-0.001487,-0.001093,0.011047,0.000148,-0.008253,-0.004208,0.008208,0.008357,0.016811
-5,0.000957,-0.004499,0.015464,-0.010009,0.008641,0.010104,-0.004238,0.013102,0.000871,0.022814,...,0.002566,-0.005486,-0.000983,0.010700,-0.001929,-0.010266,-0.002873,0.005625,0.002725,0.013268
-4,-0.007264,-0.003365,0.008239,-0.008087,0.011524,0.006667,-0.005339,0.009187,-0.000532,0.026803,...,0.005224,-0.006802,-0.001159,0.009316,-0.015802,-0.015019,-0.001616,0.003323,0.003382,0.022307
-3,-0.007825,-0.005508,0.008467,-0.012040,0.010201,0.007506,-0.003516,0.006963,0.002964,0.025913,...,0.002159,-0.010916,-0.000175,0.010927,-0.015299,-0.014121,0.001346,0.001945,0.003708,0.019947
-2,-0.004517,-0.004046,0.008836,-0.011536,0.010242,0.011459,-0.001382,0.008300,0.003522,0.026906,...,0.006973,-0.010020,0.000988,0.009600,-0.018684,-0.018624,0.002187,0.001851,0.009954,0.018830


In [16]:
# Provide plotting function with a calculated bootstrap results to plot on the graph
plot_mean_car(results_df, bootstrap_df=bootstrap_results_df, critical_value=0.99)